In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/tmdb-box-office-prediction/sample_submission.csv
/kaggle/input/tmdb-box-office-prediction/train.csv
/kaggle/input/tmdb-box-office-prediction/test.csv


In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import ast

In [3]:
Train = pd.read_csv("/kaggle/input/tmdb-box-office-prediction/train.csv")
final_Test = pd.read_csv("/kaggle/input/tmdb-box-office-prediction/test.csv")

In [4]:
# Feature Engineering
def GetCSVFromListOfDict(keyNameToFetch,column,columnName):
    column=column.copy()
    column=column.fillna('[{}]')
    columnList=[]
    for index,row in column.iteritems():
        columnStr=''
        listofDict=ast.literal_eval(row)
        for dic in listofDict:

            if(keyNameToFetch in dic.keys()):
                columnStr=columnStr+';'+str(dic[keyNameToFetch]) 
        columnStr=columnStr.strip(';') # trim leading ;
        columnList.append(columnStr)

    tempDF=pd.DataFrame(columnList,columns=[columnName])
    return tempDF[columnName] 

In [5]:
Train['belongs_to_collection']=GetCSVFromListOfDict('name',Train.belongs_to_collection,'belongs_to_collection')
Train['genres']=GetCSVFromListOfDict('name',Train.genres,'genres')
Train['production_companies']=GetCSVFromListOfDict('name',Train.production_companies,'production_companies')
Train['production_countries']=GetCSVFromListOfDict('name',Train.production_countries,'production_countries')
Train['spoken_languages']=GetCSVFromListOfDict('iso_639_1',Train.spoken_languages,'spoken_languages')
Train['Keywords']=GetCSVFromListOfDict('name',Train.Keywords,'Keywords')
Train['Crew_Dept']=GetCSVFromListOfDict('department',Train.crew,'crew')
Train['Crew_Job']=GetCSVFromListOfDict('job',Train.crew,'crew')
Train['Crew_Name']=GetCSVFromListOfDict('name',Train.crew,'crew')
Train['Crew_Gender']=GetCSVFromListOfDict('gender',Train.crew,'crew')

In [6]:
final_Test['belongs_to_collection']=GetCSVFromListOfDict('name',final_Test.belongs_to_collection,'belongs_to_collection')
final_Test['genres']=GetCSVFromListOfDict('name',final_Test.genres,'genres')
final_Test['production_companies']=GetCSVFromListOfDict('name',final_Test.production_companies,'production_companies')
final_Test['production_countries']=GetCSVFromListOfDict('name',final_Test.production_countries,'production_countries')
final_Test['spoken_languages']=GetCSVFromListOfDict('iso_639_1',final_Test.spoken_languages,'spoken_languages')
final_Test['Keywords']=GetCSVFromListOfDict('name',final_Test.Keywords,'Keywords')
final_Test['Crew_Dept']=GetCSVFromListOfDict('department',final_Test.crew,'crew')
final_Test['Crew_Job']=GetCSVFromListOfDict('job',final_Test.crew,'crew')
final_Test['Crew_Name']=GetCSVFromListOfDict('name',final_Test.crew,'crew')
final_Test['Crew_Gender']=GetCSVFromListOfDict('gender',final_Test.crew,'crew')

In [7]:
# =============================================================================
# print(len(Train.belongs_to_collection))
# Train.belongs_to_collection.value_counts()
# C = Train.loc[Train['belongs_to_collection'] == 'Singam Collection']
# =============================================================================


In [8]:
# Checking missing value in belongs_to_collection
Train['belongs_to_collection_ISMISSING']=(Train.belongs_to_collection.str.strip()=='').astype(int)
final_Test['belongs_to_collection_ISMISSING']=(final_Test.belongs_to_collection.str.strip()=='').astype(int)
cor1 =Train[['belongs_to_collection_ISMISSING','revenue']].corr()
Train.drop(columns=['belongs_to_collection'],inplace=True)
final_Test.drop(columns=['belongs_to_collection'],inplace=True)

In [9]:
# For genres column
print(len(Train.genres))
print(Train.genres.isna().sum())
Train['genres']=Train.genres.str.replace(' ','_') # so bigrams will act as unigram, and it wont become 2 columns
Train['genres']=Train.genres.str.replace(';',' ')
# Converting to 0: Absent and 1:Present

3000
0


In [10]:
from sklearn.feature_extraction.text import CountVectorizer

vectFeatures = CountVectorizer(max_features=10)
vectFeatures.fit(Train['genres'])


CountVectorizer(max_features=10)

In [11]:
featuresTrainSplit=vectFeatures.transform(Train['genres'])
featuresUnseenTestSplit=vectFeatures.transform(final_Test['genres'])
featuresTrain=pd.DataFrame(featuresTrainSplit.toarray(),columns=vectFeatures.get_feature_names())
featuresfinal_Test=pd.DataFrame(featuresUnseenTestSplit.toarray(),columns=vectFeatures.get_feature_names())
featuresTrain.columns='genres_'+featuresTrain.columns
featuresfinal_Test.columns='genres_'+featuresfinal_Test.columns
Train=pd.concat([Train,featuresTrain],axis=1)
final_Test=pd.concat([final_Test,featuresfinal_Test],axis=1)

/opt/conda/lib/python3.7/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [12]:
# Drop genres column
Train.drop(columns=['genres'],inplace=True)
final_Test.drop(columns=['genres'],inplace=True)

In [13]:
#Production Countries
print(len(Train.production_countries))
Train.production_countries.value_counts().head(20)

3000


United States of America                   1752
United Kingdom;United States of America     118
United Kingdom                               94
India                                        69
                                             55
France                                       51
Canada;United States of America              48
Germany;United States of America             47
Russia                                       46
Japan                                        31
France;United States of America              28
Canada                                       21
Australia;United States of America           21
Spain                                        19
South Korea                                  18
France;United Kingdom                        16
Australia                                    16
Japan;United States of America               16
Italy                                        14
Hong Kong                                    14
Name: production_countries, dtype: int64

In [14]:
Train['production_countries']=Train.production_countries.str.replace(' ','_') # so bigrams will act as unigram, and it wont become 2 columns
Train['production_countries']=Train.production_countries.str.replace(';',' ')
final_Test['production_countries']=final_Test.production_countries.str.replace(' ','_') # so bigrams will act as unigram, and it wont become 2 columns
final_Test['production_countries']=final_Test.production_countries.str.replace(';',' ')


In [15]:
# Creating Column films produced in America
Train['IsProductionFromUSA']=(Train['production_countries']=='united_states_of_america').astype(int)
final_Test['IsProductionFromUSA']=(final_Test['production_countries']=='united_states_of_america').astype(int)
Train.drop(columns=['production_countries'],inplace=True)
final_Test.drop(columns=['production_countries'],inplace=True)

In [16]:
#Original language and Spoken Language
Train['IsEnglishLanguage']=(
                    (Train['spoken_languages'].str.contains('en'))
                    & 
                    (Train['original_language']=='en')).astype(int)

In [17]:
final_Test['IsEnglishLanguage']=(
                    (final_Test['spoken_languages'].str.contains('en'))
                    &
                    (final_Test['original_language']=='en')).astype(int)
Train[['IsEnglishLanguage','revenue']].corr()

,IsEnglishLanguage,revenue
IsEnglishLanguage,1.000000,0.151381
revenue,0.151381,1.000000


In [18]:
Train.drop(columns=['spoken_languages','original_language'],inplace=True)
final_Test.drop(columns=['spoken_languages','original_language'],inplace=True)

In [19]:
#Keywords
Train['Keywords']=Train.Keywords.str.replace(' ','_') # so bigrams will act as unigram, and it wont become 2 columns
Train['Keywords']=Train.Keywords.str.replace(';',' ')
Train['Keywords']=Train['Keywords'].str.lower()

In [20]:
final_Test['Keywords']=final_Test.Keywords.str.replace(' ','_') # so bigrams will act as unigram, and it wont become 2 columns
final_Test['Keywords']=final_Test.Keywords.str.replace(';',' ')
final_Test['Keywords']=final_Test['Keywords'].str.lower()

In [21]:
#create columns for Keywords
from sklearn.feature_extraction.text import CountVectorizer

vectFeatures = CountVectorizer(max_features=20)
vectFeatures.fit(Train['Keywords'].str.lower())

featuresTrainSplit=vectFeatures.transform(Train['Keywords'])
featuresUnseenTestSplit=vectFeatures.transform(final_Test['Keywords'])

In [22]:
featuresTrain=pd.DataFrame(featuresTrainSplit.toarray(),columns=vectFeatures.get_feature_names())
featuresfinal_Test=pd.DataFrame(featuresUnseenTestSplit.toarray(),columns=vectFeatures.get_feature_names())

/opt/conda/lib/python3.7/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [24]:
featuresTrain.columns='Keywords'+featuresTrain.columns
featuresfinal_Test.columns='Keywords'+featuresfinal_Test.columns
Train=pd.concat([Train,featuresTrain],axis=1)
final_Test=pd.concat([final_Test,featuresfinal_Test],axis=1)

In [25]:
Train.drop(columns=['Keywords'],inplace=True)
final_Test.drop(columns=['Keywords'],inplace=True)

In [26]:
#Homepage
Train['IsHomePageAvailable']=(Train.homepage.isna()==False).astype(int)
final_Test['IsHomePageAvailable']=(final_Test.homepage.isna()==False).astype(int)

In [27]:
Train[['IsHomePageAvailable','revenue']].corr()

,IsHomePageAvailable,revenue
IsHomePageAvailable,1.000000,0.263179
revenue,0.263179,1.000000


In [28]:
#Train Date
dateSplit=Train.release_date.str.extract('([0-9]+)/([0-9]+)/([0-9]+)')
dateSplit.columns=['ReleaseMonth','ReleaseDate','ReleaseYear']

In [29]:
dateSplit.loc[dateSplit.ReleaseYear.astype(int)>20,'ReleaseYear']='19'+dateSplit.loc[dateSplit.ReleaseYear.astype(int)>20,'ReleaseYear']
dateSplit.loc[dateSplit.ReleaseYear.astype(int)<=20,'ReleaseYear']='20'+dateSplit.loc[dateSplit.ReleaseYear.astype(int)<=20,'ReleaseYear']

In [30]:
Train.drop(columns=['release_date'],inplace=True)
Train=pd.concat([Train,dateSplit.astype(int)],axis=1)

In [31]:
# Test Date
print(final_Test.release_date.mode())
final_Test['release_date'].fillna('9/9/11',inplace=True)

0    9/9/11
dtype: object


In [32]:
dateSplit1=final_Test.release_date.str.extract('([0-9]+)/([0-9]+)/([0-9]+)')
dateSplit1.columns=['ReleaseMonth','ReleaseDate','ReleaseYear']

In [33]:
dateSplit1.loc[dateSplit1.ReleaseYear.astype(int)>20,'ReleaseYear']='19'+dateSplit1.loc[dateSplit1.ReleaseYear.astype(int)>20,'ReleaseYear']
dateSplit1.loc[dateSplit1.ReleaseYear.astype(int)<=20,'ReleaseYear']='20'+dateSplit1.loc[dateSplit1.ReleaseYear.astype(int)<=20,'ReleaseYear']


In [34]:
final_Test.drop(columns=['release_date'],inplace=True)
final_Test=pd.concat([final_Test,dateSplit1.astype(int)],axis=1)

In [35]:
# Log Scaling
Train['revenue']=np.log1p(Train.revenue)

In [36]:
Train['budget']=np.log1p(Train.budget)
final_Test['budget']=np.log1p(final_Test.budget)

In [37]:
Train['popularity']=np.log1p(Train.popularity)
final_Test['popularity']=np.log1p(final_Test.popularity)

In [38]:
#Model
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
TrainNum=Train.select_dtypes(include=numerics)
final_TestNum=final_Test.select_dtypes(include=numerics)
TrainNum.drop(columns=['id'],inplace=True)
final_TestNum.drop(columns=['id'],inplace=True)

/opt/conda/lib/python3.7/site-packages/pandas/core/frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [39]:
TrainNum=TrainNum.fillna(TrainNum.median())
final_TestNum=final_TestNum.fillna(TrainNum.median())

In [40]:
#Train Test split
from sklearn import model_selection # for splitting into train and test
import sklearn
# Split-out validation dataset
X = TrainNum.drop(columns=['revenue'])
Y = TrainNum['revenue']

In [41]:
validation_size = 0.2
seed = 100
X_train, X_test, Y_train, Y_test = sklearn.model_selection.train_test_split(X, Y, test_size=validation_size, random_state=seed)

In [42]:
#Import XG Boost
import xgboost
model_XG = xgboost.XGBRegressor() 
model_XG.fit(X_train, Y_train)

XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
             colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
             early_stopping_rounds=None, enable_categorical=False,
             eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
             importance_type=None, interaction_constraints='',
             learning_rate=0.300000012, max_bin=256, max_cat_to_onehot=4,
             max_delta_step=0, max_depth=6, max_leaves=0, min_child_weight=1,
             missing=nan, monotone_constraints='()', n_estimators=100, n_jobs=0,
             num_parallel_tree=1, predictor='auto', random_state=0, reg_alpha=0,
             reg_lambda=1, ...)

In [43]:
# make predictions for test data

trainResult_XG = model_XG.predict(X_train)
testResult_XG = model_XG.predict(X_test)
final_TestResult_XG=model_XG.predict(final_TestNum)

In [44]:
########## TRAIN DATA RESULT ##########

print('---------- TRAIN DATA RESULT ----------')
# The mean squared error
print("Mean squared error: %.5f"%np.sqrt( mean_squared_error(Y_train, trainResult_XG)))
# Explained variance score: 1 is perfect prediction
print('Variance score: %.4f' % r2_score(Y_train, trainResult_XG))

---------- TRAIN DATA RESULT ----------
Mean squared error: 0.47408
Variance score: 0.9759


In [45]:
########## TEST DATA RESULT ##########

print('---------- TEST DATA RESULT ----------')
# The mean squared error
print("Mean squared error: %.5f"% np.sqrt(mean_squared_error(Y_test, testResult_XG)))
# Explained variance score: 1 is perfect prediction
print('Variance score: %.4f' % r2_score(Y_test, testResult_XG))

---------- TEST DATA RESULT ----------
Mean squared error: 2.25853
Variance score: 0.4624


In [46]:
#############Submission##################

final_TestResult_XG=np.expm1(final_TestResult_XG)
submission=pd.DataFrame([final_Test.id,final_TestResult_XG]).T

submission.columns=['id','revenue']

submission.id=submission.id.astype(int)

submission.to_csv('submission.csv',index=False)